# Model Comparison Lab

In this lab we will compare the performance of all the models we have learned about so far, using the car evaluation dataset.

## 1. Prepare the data

The [car evaluation dataset](https://archive.ics.uci.edu/ml/machine-learning-databases/car/) is in the assets/datasets folder. By now you should be very familiar with this dataset.

1. Load the data into a pandas dataframe
- Encode the categorical features properly: define a map that preserves the scale (assigning smaller numbers to words indicating smaller quantities)
- Separate features from target into X and y

In [95]:
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.cross_validation import train_test_split, cross_val_score, KFold
from sklearn.pipeline import Pipeline, make_pipeline, make_union
from sklearn import metrics
from sklearn.preprocessing import Imputer, StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from IPython.display import Image
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import roc_auc_score, classification_report
from sqlalchemy import create_engine
import patsy
from sklearn import neighbors, metrics
import seaborn as sns
sns.set(color_codes=True)
import os
from sklearn import svm
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import subprocess
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
import scipy
from imdbpie import Imdb
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
import requests
import nltk
import matplotlib.pyplot as plt
import urllib
from bs4 import BeautifulSoup
import nltk
import collections
import re
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.externals.six import StringIO  
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import confusion_matrix, classification_report, mean_squared_error
import pandas as pd
import numpy as np
from sklearn.cross_validation import cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from sklearn.externals.six import StringIO
from sklearn.tree import export_graphviz
import pydot
import patsy
from IPython.display import Image
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
%matplotlib inline

In [6]:
df = pd.read_csv('/Users/smoot/Desktop/ga/DSI_SM_01/curriculum/week-06/3.3-lesson/assets/datasets/car.csv')
df.head()

,buying,maint,doors,persons,lug_boot,safety,acceptability
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [27]:
df.acceptability.unique()

array(['unacc', 'acc', 'vgood', 'good'], dtype=object)

In [100]:
le = LabelEncoder()
y = le.fit_transform(df['acceptability'])
# X = patsy.dmatrix('buying + maint + doors + persons + lug_boot + safety', df)
features = [c for c in df.columns if c != 'acceptability']
X_temp = df[features]
X = pd.get_dummies(X_temp)
X_test, X_train, y_test, y_train = train_test_split(X, y, train_size = .33, stratify = y, random_state = 42)
print X

      buying_high  buying_low  buying_med  buying_vhigh  maint_high  \
0             0.0         0.0         0.0           1.0         0.0   
1             0.0         0.0         0.0           1.0         0.0   
2             0.0         0.0         0.0           1.0         0.0   
3             0.0         0.0         0.0           1.0         0.0   
4             0.0         0.0         0.0           1.0         0.0   
5             0.0         0.0         0.0           1.0         0.0   
6             0.0         0.0         0.0           1.0         0.0   
7             0.0         0.0         0.0           1.0         0.0   
8             0.0         0.0         0.0           1.0         0.0   
9             0.0         0.0         0.0           1.0         0.0   
10            0.0         0.0         0.0           1.0         0.0   
11            0.0         0.0         0.0           1.0         0.0   
12            0.0         0.0         0.0           1.0         0.0   
13    

In [50]:
# Recursive Feature Elimination
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
# load the iris datasets
# create a base classifier used to evaluate a subset of attributes
model = LogisticRegression()
# create the RFE model and select 3 attributes
rfe = RFE(model, 3)
rfe = rfe.fit(X, y)
# summarize the selection of the attributes
print(rfe.support_)
print(rfe.ranking_)


[False False False False False False False False False False  True  True
 False False  True False]
[ 2  5  6  8  9 10  3 14 13 12  1  1 11  7  1  4]


## 2. Useful preparation

Since we will compare several models, let's write a couple of helper functions.

1. Separate X and y between a train and test set, using 30% test set, random state = 42
    - make sure that the data is shuffled and stratified
2. Define a function called `evaluate_model`, that trains the model on the train set, tests it on the test, calculates:
    - accuracy score
    - confusion matrix
    - classification report
3. Initialize a global dictionary to store the various models for later retrieval


## 3.a KNN

Let's start with `KNeighborsClassifier`.

1. Initialize a KNN model
- Evaluate it's performance with the function you previously defined
- Find the optimal value of K using grid search
    - Be careful on how you perform the cross validation in the grid search

In [39]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
print knn.score(X_test, y_test)
y_pred = knn.predict(X_test)

0.861403508772


In [40]:
print classification_report(y_test, y_pred)

             precision    recall  f1-score   support

          0       0.73      0.75      0.74       127
          1       0.33      0.17      0.23        23
          2       0.92      0.96      0.94       399
          3       0.64      0.33      0.44        21

avg / total       0.85      0.86      0.85       570



In [43]:
cm = confusion_matrix(y_test, y_pred)
print cm

[[ 95   2  28   2]
 [ 15   4   2   2]
 [ 11   3 385   0]
 [  9   3   2   7]]


In [106]:

gs = GridSearchCV(knn, dict(n_neighbors=range(1,26)), cv = 5)
gs.fit(X_train, y_train)
gs.score(X_test, y_test)

0.89473684210526316

In [107]:
gs.best_params_

{'n_neighbors': 5}

## 3.b Bagging + KNN

Now that we have found the optimal K, let's wrap `KNeighborsClassifier` in a BaggingClassifier and see if the score improves.

1. Wrap the KNN model in a Bagging Classifier
- Evaluate performance
- Do a grid search only on the bagging classifier params

In [108]:
knn_bag = BaggingClassifier(knn)
knn_bag.fit(X_train, y_train)
bag_p = knn_bag.predict(X_test)
knn_bag.score(X_test, y_test)

0.91929824561403506

In [109]:
bagging_params = {
    'n_estimators': [10, 20],
    'max_samples' : [0.7, 1.0],
    'max_features': [0.7, 1.0],
    'bootstrap_features': [True, False]
}
csg = GridSearchCV(knn_bag, bagging_params, n_jobs = -1, cv=KFold(len(y), shuffle=True))
csg.fit(X_train, y_train)
csg.score(X_test, y_test)

JoblibIndexError: JoblibIndexError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/Users/smoot/anaconda/lib/python2.7/runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    157     pkg_name = mod_name.rpartition('.')[0]
    158     main_globals = sys.modules["__main__"].__dict__
    159     if alter_argv:
    160         sys.argv[0] = fname
    161     return _run_code(code, main_globals, None,
--> 162                      "__main__", fname, loader, pkg_name)
        fname = '/Users/smoot/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = 'ipykernel'
    163 
    164 def run_module(mod_name, init_globals=None,
    165                run_name=None, alter_sys=False):
    166     """Execute a module's code without importing it

.................................

In [112]:
import xgboost as xgb


In [110]:
print classification_report(y_test, bag_p)

             precision    recall  f1-score   support

          0       0.84      0.81      0.83       127
          1       0.85      0.48      0.61        23
          2       0.94      0.99      0.97       399
          3       0.89      0.76      0.82        21

avg / total       0.92      0.92      0.92       570



In [55]:
confusion_matrix(y_test, bag_p)

array([[ 92,   1,  32,   2],
       [ 17,   2,   2,   2],
       [ 19,   2, 378,   0],
       [  7,   1,   5,   8]])

In [57]:
lr = LogisticRegression()
lr.fit(X_test, y_test)
lry_pred = lr.predict(X_test)
lr.score(X_test, y_test)

0.87368421052631584

## 4. Logistic Regression

Let's see if logistic regression performs better

1. Initialize LR and test on Train/Test set
- Find optimal params with Grid Search
- See if Bagging improves the score

In [58]:
lr_bag = BaggingClassifier(lr)
lr_bag.fit(X_train, y_train)
lr_bag_p = lr_bag.predict(X_test)
lr_bag.score(X_test, y_test)

0.86491228070175441

In [59]:
print classification_report(y_test, lr_bag_p)

             precision    recall  f1-score   support

          0       0.66      0.84      0.74       127
          1       0.50      0.13      0.21        23
          2       0.95      0.95      0.95       399
          3       0.71      0.24      0.36        21

avg / total       0.86      0.86      0.85       570



In [61]:
confusion_matrix(y_test, lr_bag_p)

array([[107,   2,  18,   0],
       [ 18,   3,   0,   2],
       [ 20,   1, 378,   0],
       [ 16,   0,   0,   5]])

## 5. Decision Trees

Let's see if Decision Trees perform better

1. Initialize DT and test on Train/Test set
- Find optimal params with Grid Search
- See if Bagging improves the score

In [63]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dec_t_p = dt.predict(X_test)
dt.score(X_test, y_test)

0.89473684210526316

In [64]:
dt_bag = BaggingClassifier(DecisionTreeClassifier())
dt_bag.fit(X_train, y_train)
dec_t_p = dt_bag.predict(X_test)
dt_bag.score(X_test, y_test)

0.9140350877192982

## 6. Support Vector Machines

Let's see if SVM perform better

1. Initialize SVM and test on Train/Test set
- Find optimal params with Grid Search
- See if Bagging improves the score

In [67]:
sv = svm.SVC()
sv.fit(X_train, y_train)
sv_p = sv.predict(X_test)
sv.score(X_test, y_test)

0.85964912280701755

In [71]:
sv_bag = svm.SVC()
sv_bag.fit(X_train, y_train)
sv_bag_p = sv_bag.predict(X_test)
sv_bag.score(X_test, y_test)

0.85964912280701755

In [80]:
print 'svm model: ', sv.score(X_test, y_test)
print 'svm with bagging: ', sv_bag.score(X_test, y_test)
print 'knn model: ', knn.score(X_test, y_test)
print 'knn with bagging model: ', knn_bag.score(X_test, y_test)
print 'decision tree: ', dt.score(X_test, y_test)
print 'decision tree with bagging: ', dt_bag.score(X_test, y_test)
print 'logitstic regression: ', lr.score(X_test, y_test)
print 'logistic regression with bagging: ', lr_bag.score(X_test, y_test)
print 'random forest: ', rf.score(X_test, y_test)
print 'extra trees: ', rf_e.score(X_test, y_test)

svm model:  0.859649122807
svm with bagging:  0.859649122807
knn model:  0.861403508772
knn with bagging model:  0.842105263158
decision tree:  0.894736842105
decision tree with bagging:  0.914035087719
logitstic regression:  0.873684210526
logistic regression with bagging:  0.864912280702
random forest:  0.875438596491
extra trees:  0.878947368421


## 7. Random Forest & Extra Trees

Let's see if Random Forest and Extra Trees perform better

1. Initialize RF and ET and test on Train/Test set
- Find optimal params with Grid Search

In [75]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf_p = rf.predict(X_test)
rf.score(X_test, y_test)

0.87543859649122802

In [79]:
rf_e = ExtraTreesClassifier()
rf_e.fit(X_train, y_train)
rf_p_e = rf_e.predict(X_test)
rf_e.score(X_test, y_test)

0.87894736842105259

In [81]:
from sklearn.cross_validation import train_test_split, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

def evaluate_model(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    a = accuracy_score(y_test, y_pred)
    
    cm = confusion_matrix(y_test, y_pred)
    cr = classification_report(y_test, y_pred)
    
    print cm
    print cr
    
    return a

all_models = {}

## 8. Model comparison

Let's compare the scores of the various models.

1. Do a bar chart of the scores of the best models. Who's the winner on the train/test split?
- Re-test all the models using a 3 fold stratified shuffled cross validation
- Do a bar chart with errorbars of the cross validation average scores. is the winner the same?


## Bonus

We have encoded the data using a map that preserves the scale.
Would our results have changed if we had encoded the categorical data using `pd.get_dummies` or `OneHotEncoder`  to encode them as binary variables instead?

1. Repeat the analysis for this scenario. Is it better?
- Experiment with other models or other parameters, can you beat your classmates best score?